In [6]:
import tensorflow as tf
import numpy as np
import sklearn
import random,os,io
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense

In [3]:
from tensorflow.keras.callbacks import Callback, EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorboard.plugins.hparams import api as hp

In [4]:
import pandas as pd
dep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XTr.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_train=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yTr.dat",sep='\s+',names=['target'])
dep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XV.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_val=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yV.dat",sep='\s+',names=["target"])
dep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\XT.dat",sep='\s+',names=[str(i) for i in range(0,16)])
indep_test=pd.read_csv("D:\\bro\\Analysis\\mSANN\\cd\\yT.dat",sep='\s+',names=["target"])

dep_train.shape,indep_train.shape,dep_val.shape,indep_val.shape,dep_test.shape,indep_test.shape

((42039, 16), (42039, 1), (10935, 16), (10935, 1), (10934, 16), (10934, 1))

In [5]:
# Hyper-parameters
batch_size = 50
num_hidden_1 = 12
num_hidden_2 = 12
learning_rate = 0.0001
training_epochs = 100
dropout_keep_prob = 0.3 # set to no dropout by default

# variable to control the resolution at which the training results are stored
display_step = 1

In [9]:
reg = tf.keras.Sequential()
reg.add(Dense(units = 6, activation = 'relu', input_dim = 16))

In [11]:
reg.add(Dense(units = 6, activation = 'relu'))

In [12]:
reg.add(Dense(units = 6, activation = 'relu'))

In [13]:
reg.add(Dense(units = 1, activation = 'linear'))

In [14]:
reg.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 102       
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 6)                 42        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [16]:
optimizer = tf.keras.optimizers.Adam()
reg.compile(optimizer=optimizer,  
            loss='mean_absolute_error')

In [19]:
reg.fit(dep_train, indep_train, batch_size = 10, epochs = 10)

Epoch 1/10
4204/4204 [==============================] - 18s 4ms/step - loss: 0.0013
Epoch 2/10
4204/4204 [==============================] - 15s 4ms/step - loss: 8.0503e-04
Epoch 3/10
4204/4204 [==============================] - 15s 4ms/step - loss: 6.6461e-04
Epoch 4/10
4204/4204 [==============================] - 15s 4ms/step - loss: 6.3509e-04
Epoch 5/10
4204/4204 [==============================] - 16s 4ms/step - loss: 6.0914e-04
Epoch 6/10
4204/4204 [==============================] - 15s 4ms/step - loss: 6.0720e-04
Epoch 7/10
4204/4204 [==============================] - 16s 4ms/step - loss: 5.8537e-04
Epoch 8/10
4204/4204 [==============================] - 16s 4ms/step - loss: 5.7827e-04
Epoch 9/10
4204/4204 [==============================] - 17s 4ms/step - loss: 5.5158e-04
Epoch 10/10
4204/4204 [==============================] - 15s 4ms/step - loss: 5.4294e-04


In [20]:
reg.save("demo")

INFO:tensorflow:Assets written to: demo\assets


In [21]:
reg_reload=tf.keras.models.load_model("demo")
reg_reload.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 102       
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 6)                 42        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


In [22]:
reg_reload.predict(dep_test)

342/342 [==============================] - 2s 4ms/step


array([[0.01661042],
       [0.01399928],
       [0.01650758],
       ...,
       [0.01186464],
       [0.01621098],
       [0.00988809]], dtype=float32)

In [42]:
class SobolevNetwork(Model):
    def __init__(self, input_dim, num_hidden,init = None):
        super(SobolevNetwork, self).__init__()
        self.input_dim = input_dim 
        self.num_hidden = num_hidden
        self.W1 = tf.Variable(tf.random.normal([self.input_dim, self.num_hidden],stddev=0.1))
        self.b1 = tf.Variable(tf.ones([self.num_hidden]))
        self.W2 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b2 = tf.Variable(tf.ones([self.num_hidden]))
        self.W3 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b3 = tf.Variable(tf.ones([self.num_hidden]))
        self.W4 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b4 = tf.Variable(tf.ones([self.num_hidden]))
        self.W5 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b5 = tf.Variable(tf.ones([self.num_hidden]))
        self.W6 = tf.Variable(tf.random.normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b6 = tf.Variable(tf.ones([self.num_hidden]))        
        self.W7 = tf.Variable(tf.random.normal([self.num_hidden, 1],stddev=0.1))
        self.b7 = tf.Variable(tf.ones([1]))
        self.w = [(self.W1, self.b1), (self.W2, self.b2), (self.W3, self.b3),(self.W4, self.b4), (self.W5, self.b5), (self.W6, self.b6),(self.W7, self.b7)]
        
    def call(self, X):
        #Input layer
        out = X
        #Hidden layers
        W,b = self.w[0]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.w[1]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.w[2]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.w[3]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.w[4]
        out = tf.nn.leaky_relu(tf.matmul(out, W) + b)
        W,b = self.w[5]
        out = tf.nn.relu(tf.matmul(out, W) + b)
        #Output layer
        W,b = self.w[-1]
        out = tf.matmul(out, W) + b
        return out


In [43]:
custom_snn= SobolevNetwork(input_dim=16,num_hidden=num_hidden_1)

In [44]:
custom_snn.compile(optimizer=optimizer,  
            loss='mean_absolute_error')

In [45]:
custom_snn.fit(dep_train, indep_train, batch_size = 10, epochs = 2)

Epoch 1/2
4204/4204 [==============================] - 23s 5ms/step - loss: 0.0049
Epoch 2/2
4204/4204 [==============================] - 19s 4ms/step - loss: 0.0018
